In [1]:
import pandas as pd # import Pandas library
import gzip # to manage gzip-compressed files
import json # to handle json data

In [3]:
def convert_single_to_double_quotes(line):
    """Safely converts single quotes in a string to double quotes for JSON parsing."""
    try:
        return json.dumps(eval(line)) #eval first the incorrect json format of simple quote, into a python dictionary, since python
                                      #actually uses simple in dictionaries.
                                      #Then, json.dumps converts that string (dictionary) into a json with a real correct format of
                                      #two quotation marks
    except:
        return None

In [4]:
# List to store the converted lines
converted_lines = []

# Open the compressed file and read line by line with utf-8 encoding
with gzip.open('./datasets_crudos/user_reviews.json.gz', 'rt', encoding='utf-8') as file:
    for line in file:
        converted_line = convert_single_to_double_quotes(line)
        if converted_line:
            converted_lines.append(converted_line)

# Load the converted lines into a DataFrame
user_reviews = pd.DataFrame([json.loads(line) for line in converted_lines])

In [5]:
user_reviews1 = user_reviews.explode(["reviews"]) # use explode to separate the data inside reviews column

In [6]:
user_reviews1.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


In [8]:
user_reviews2 = pd.json_normalize(user_reviews1['reviews']).set_index(user_reviews1['reviews'].index)

In [9]:
user_reviews2.head()

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [10]:
user_reviews3= pd.concat([user_reviews2, user_reviews1], axis = 1)

In [37]:
user_reviews3.columns

Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', 'user_id', 'user_url', 'reviews'],
      dtype='object')

In [38]:
# delete columns funny, posted, last_edited, helpful, user_url, reviews

user_reviews4 = user_reviews3.drop(['funny', 'posted','last_edited','helpful','user_url', 'reviews'], axis=1)

In [39]:
user_reviews4.columns # check update

Index(['item_id', 'recommend', 'review', 'user_id'], dtype='object')

In [40]:
# create a joined_id between user_id & item_id to joint user_reviews table with user_items table and steam_games table.
user_reviews4['joined_id'] = user_reviews4['user_id'] + user_reviews4['item_id']

In [46]:
user_reviews4.reset_index(drop=True)

,item_id,recommend,review,user_id,joined_id
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,765611979709824791250
1,22200,True,It's unique and worth a playthrough.,76561197970982479,7656119797098247922200
2,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,7656119797098247943110
3,251610,True,I know what you think when you see this title ...,js41637,js41637251610
4,227300,True,For a simple (it's actually not all that simpl...,js41637,js41637227300
...,...,...,...,...,...
59328,70,True,a must have classic from steam definitely wort...,76561198312638244,7656119831263824470
59329,362890,True,this game is a perfect remake of the original ...,76561198312638244,76561198312638244362890
59330,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,LydiaMorley273110
59331,730,True,:D,LydiaMorley,LydiaMorley730


In [ ]:
user_reviews4.head(50)

In [48]:
user_reviews4['joined_id'][300]

300       76561198069560699550
300    76561198069560699259080
300     7656119806956069972850
300      765611980695606998870
Name: joined_id, dtype: object

In [51]:

print(user_reviews4.isna().sum()) # check for na values

item_id      28
recommend    28
review       28
user_id       0
joined_id    28
dtype: int64


In [50]:
user_reviews4.dropna(how='all', inplace=True) # the how=all ensures to drop only the rows that are completely empty

In [52]:
# Export DataFrame to Excel file to check
file_name = './dataset_ready/users_reviews_v2.xlsx' #file name included extension
pd.DataFrame(user_reviews4, columns = user_reviews4.columns).to_excel(file_name, index=False)  # "index=False" to avoid indexs columns

In [53]:
# Export DataFrame to csv file to check
file_name = './dataset_ready/users_reviews_v2.csv' #file name included extension
pd.DataFrame(user_reviews4, columns = user_reviews4.columns).to_csv(file_name, index=False)  # "index=False" to avoid indexs columns